In [1]:
import shutil
import pandas as pd
import numpy as np
import os
import transformers
from transformers import  AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
from datasets import load_dataset
from sklearn.model_selection import train_test_split 

# the name of the pre-trained model we want to use
MODEL_NAME = "bert-base-uncased" 

c:\Users\Wenhao\miniconda3\envs\deep_wa_bert\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data Loading and Preprocessing

In [2]:
data = pd.read_parquet("../labeled_post.parquet")
data.shape

(80571, 15)

### Creating the label

In [3]:
flairs = {
    'Not the A-hole': "NTA",
    'Asshole': "YTA",
#     'No A-holes here': "NAH",
#     'Everyone Sucks': "ESH",
#     'Not enough info': "INFO",
    # 'UPDATE': "?",
    # 'TL;DR': "?",
    'not the a-hole': "NTA",
    # 'POO Mode Activated 💩',
    'asshole': "YTA",
    # '': "?",
    # 'META': "?",
#     'Shitpost': "?",
    'not the asshole': "NTA",
#     'no a--holes here': "NAH",
#     'everyone sucks': "ESH",
#     'too close to call': "?",
#     'not enough info': "INFO",
}

# filter data and label verdicts
labeled_data = data.loc[data.link_flair_text.isin(flairs)].copy()
labeled_data["verdict"] = labeled_data.link_flair_text.map(flairs)
labeled_data["verdict"] = (labeled_data["verdict"] == "YTA").astype(int)
labeled_data["verdict"].value_counts()

0    57633
1    14135
Name: verdict, dtype: int64

In [4]:
negative_data = labeled_data[labeled_data["verdict"] == 0]
positive_data = labeled_data[labeled_data["verdict"] == 1]
negative_data = negative_data.sample(len(positive_data), random_state=547)

In [5]:
labeled_data = pd.concat([negative_data, positive_data])

### Creating the textual input

In [6]:
from gensim.parsing import remove_stopwords, strip_numeric, strip_punctuation, strip_multiple_whitespaces

labeled_data["title_text"] = labeled_data["title"] + " --- " + labeled_data["selftext"]
labeled_data["title_text"] = labeled_data["title_text"].str.lower()
labeled_data["title_text"] = labeled_data["title_text"].apply(strip_multiple_whitespaces)
labeled_data = labeled_data.replace(to_replace=[''], value=np.nan).dropna(subset=["title_text", "verdict"])

In [7]:
final_data = labeled_data[["title_text", "verdict"]]
final_data.head()

,title_text,verdict
15509,aita for not stopping my little brother from d...,0
54638,aita for setting my little brother into a fit ...,0
21845,wibta if i called out my landlord? --- i (29f)...,0
13497,aita for not feeding my cousin’s dog? --- my f...,0
26178,aita for telling my step dad i no longer want ...,0


### Data splitting

In [8]:
train, val = train_test_split(final_data, test_size=0.2, random_state=547)
train.to_csv("./data/train.csv", index=False)
val.to_csv("./data/val.csv", index=False)

In [9]:
dataset = load_dataset("csv", data_files={"train": "./data/train.csv", "val": "./data/val.csv"})

Using custom data configuration default-db78e356e0e09e10


Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 443.65it/s]


Dataset csv downloaded and prepared to C:\Users\Wenhao\.cache\huggingface\datasets\csv\default-db78e356e0e09e10\0.0.0\652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


100%|██████████| 2/2 [00:00<00:00, 363.44it/s]


## Tokenization

In [10]:
from transformers import AutoTokenizer

### Preprocess Setup ###
# tokenization hyperparameters
PADDING = 'max_length' # padding strategy
PADDING_SIDE = 'right' # the side on which the model should have padding applied
TRUNCATION = True # truncate strategy
TRUNCATION_SIDE = 'right' # the side on which the model should have truncation applied
MAX_LEN = 512 # maximum length to use by one of the truncation/padding parameters

# Load the pre-trained tokenmizer ###
TOKENIZER = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    padding_side=PADDING_SIDE,
    truncation_side=TRUNCATION_SIDE,
)

# Define the preprocess function ###
def preprocess_function(examples):
    """
    Preprocess the description field
    ---
    Arguments:
    examples (str, List[str], List[List[str]]: the sequence or batch of sequences to be encoded/tokenized

    Returns:
    tokenized (transformers.BatchEncoding): tokenized descriptions 
    """
    tokenized = TOKENIZER(
        examples["title_text"],
        padding=PADDING,
        truncation=TRUNCATION,
        max_length=MAX_LEN,
    )

    return tokenized

In [11]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns("title_text")

Parameter 'function'=<function preprocess_function at 0x000001BD94CEDB40> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
100%|██████████| 6/6 [00:01<00:00,  4.35ba/s]


In [12]:
tokenized_dataset = tokenized_dataset.rename_column("verdict", "label")

## Evaluation

In [13]:
import numpy as np
from datasets import load_metric
from sklearn.metrics import matthews_corrcoef, cohen_kappa_score, confusion_matrix, balanced_accuracy_score

### Evaluation Metrics ###
metric_acc = load_metric("accuracy")
metric_f1 = load_metric("f1")
metric_precision = load_metric("precision")
metric_recall = load_metric("recall")
metric_auc = load_metric("roc_auc")

def compute_metrics(eval_pred):
    """
    Compute the metrics 
    ---
    Arguments:
    eval_pred (tuple): the predicted logits and truth labels

    Returns:
    metrics (dict{str: float}): contains the computed metrics 
    """
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    prediction_scores = np.max(logits, axis=-1)
    print(logits.shape, labels.shape)
    print(predictions.shape, prediction_scores.shape)

    pred_true = np.count_nonzero(predictions)
    pred_false = predictions.shape[0] - pred_true
    actual_true = np.count_nonzero(labels)
    actual_false = labels.shape[0] - actual_true

    acc = metric_acc.compute(predictions=predictions, references=labels)['accuracy']
    f1 = metric_f1.compute(predictions=predictions, references=labels)['f1']
    precision = metric_precision.compute(predictions=predictions, references=labels)['precision']
    recall = metric_recall.compute(predictions=predictions, references=labels)['recall']
    roc_auc = metric_auc.compute(prediction_scores=predictions, references=labels)['roc_auc']
    matthews_correlation = matthews_corrcoef(y_true=labels, y_pred=predictions)
    cohen_kappa = cohen_kappa_score(y1=labels, y2=predictions)
    balanced_accuracy = balanced_accuracy_score(y_true=labels, y_pred=predictions)

    tn, fp, fn, tp = confusion_matrix(y_true=labels, y_pred=predictions).ravel()
    specificity = tn / (tn + fp)
    sensitivity = tp / (tp + fn)
    informedness = specificity + sensitivity - 1

    metrics = {
        "pred_true": pred_true,
        "pred_false": pred_false,
        "actual_true": actual_true,
        "actual_false": actual_false,
        "accuracy": acc,
        "f1_score": f1,
        "precision": precision,
        "recall": recall,
        "roc_auc": roc_auc,
        "matthews_correlation": matthews_correlation,
        "cohen_kappa": cohen_kappa,
        "true_negative": tn,
        "false_positive": fp,
        "false_negative": fn,
        "true_positive": tp,
        "specificity": specificity,
        "sensitivity": sensitivity,
        "informedness": informedness,
        "balanced_accuracy": balanced_accuracy
    }
    return metrics

## Model fitting

### Hyperparameters

In [15]:
# model hyperparameters
CLASSIFIER_DROPOUT = 0.15 # dropout ratio for the classification head
NUM_CLASSES = 2 # number of classes

# optimization hyperparameters ###
SEED = 547 # random seed for splitting the data into batches
BATCH_SIZE = 16 # batch size for both training and evaluation
GRAD_ACC_STEPS = 4 # number of steps for gradient accumulation
LR = 5e-5 # initial learning rate
WEIGHT_DECAY = 2e-3 # weight decay to apply in the AdamW optimizer
EPOCHS = 10 # total number of training epochs 
LR_SCHEDULER = "cosine" # type of learning rate scheduler
STRATEGY = "steps" # strategy for logging, evaluation, and saving
STEPS = 200 # number of steps for logging, evaluation, and saving
EVAL_METRIC = "f1_score" # metric for selecting the best model

### Fitting the model

In [16]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    classifier_dropout=CLASSIFIER_DROPOUT,
    num_labels=NUM_CLASSES,
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [17]:
# set up the training arguments
training_args = TrainingArguments(
    output_dir="./model",
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACC_STEPS,
    learning_rate=LR,
    weight_decay=WEIGHT_DECAY, 
    num_train_epochs=EPOCHS,
    lr_scheduler_type=LR_SCHEDULER,
    evaluation_strategy=STRATEGY,
    logging_strategy=STRATEGY, 
    save_strategy=STRATEGY,
    eval_steps=STEPS,
    logging_steps=STEPS,
    save_steps=STEPS,
    seed=SEED,
    load_best_model_at_end=True,
    metric_for_best_model=EVAL_METRIC,
    report_to="none"
)

In [18]:
# set up the trainer 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['val'],
    tokenizer=TOKENIZER,   
    compute_metrics=compute_metrics,
)

In [19]:
try:
    shutil.rmtree("./model") # remove possible cache
    # shutil.rmtree(best_model_dir)
    # os.remove(best_model_dir_zip)
    os.mkdir("./model")
except:
    None

In [20]:
trainer.train()

c:\Users\Wenhao\miniconda3\envs\deep_wa_bert\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 22616
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 4
  Total optimization steps = 3530
  6%|▌         | 200/3530 [04:34<1:17:21,  1.39s/it]***** Running Evaluation *****
  Num examples = 5654
  Batch size = 16


{'loss': 0.6919, 'learning_rate': 4.960502152176574e-05, 'epoch': 0.57}


                                                    
  6%|▌         | 200/3530 [05:20<1:17:21,  1.39s/it]Saving model checkpoint to ./model\checkpoint-200
Configuration saved in ./model\checkpoint-200\config.json


(5654, 2) (5654,)
(5654,) (5654,)
{'eval_loss': 0.6858039498329163, 'eval_pred_true': 1531, 'eval_pred_false': 4123, 'eval_actual_true': 2803, 'eval_actual_false': 2851, 'eval_accuracy': 0.5680933852140078, 'eval_f1_score': 0.4365482233502538, 'eval_precision': 0.6178967994774657, 'eval_recall': 0.33749554049232966, 'eval_roc_auc': 0.566152189748094, 'eval_matthews_correlation': 0.14886414763463218, 'eval_cohen_kappa': 0.1328117374769574, 'eval_true_negative': 2266, 'eval_false_positive': 585, 'eval_false_negative': 1857, 'eval_true_positive': 946, 'eval_specificity': 0.7948088390038583, 'eval_sensitivity': 0.33749554049232966, 'eval_informedness': 0.132304379496188, 'eval_balanced_accuracy': 0.566152189748094, 'eval_runtime': 45.5841, 'eval_samples_per_second': 124.034, 'eval_steps_per_second': 7.766, 'epoch': 0.57}


Model weights saved in ./model\checkpoint-200\pytorch_model.bin
tokenizer config file saved in ./model\checkpoint-200\tokenizer_config.json
Special tokens file saved in ./model\checkpoint-200\special_tokens_map.json
 11%|█▏        | 400/3530 [10:06<1:14:32,  1.43s/it] ***** Running Evaluation *****
  Num examples = 5654
  Batch size = 16


...

                                                     
 96%|█████████▋| 3400/3530 [1:37:30<03:13,  1.49s/it]Saving model checkpoint to ./model\checkpoint-3400
Configuration saved in ./model\checkpoint-3400\config.json


(5654, 2) (5654,)
(5654,) (5654,)
{'eval_loss': 2.4857797622680664, 'eval_pred_true': 2988, 'eval_pred_false': 2666, 'eval_actual_true': 2803, 'eval_actual_false': 2851, 'eval_accuracy': 0.623452423063318, 'eval_f1_score': 0.632360559488862, 'eval_precision': 0.6127844712182061, 'eval_recall': 0.6532286835533357, 'eval_roc_auc': 0.6237030825693721, 'eval_matthews_correlation': 0.24779943172486887, 'eval_cohen_kappa': 0.24726878247098194, 'eval_true_negative': 1694, 'eval_false_positive': 1157, 'eval_false_negative': 972, 'eval_true_positive': 1831, 'eval_specificity': 0.5941774815854086, 'eval_sensitivity': 0.6532286835533357, 'eval_informedness': 0.24740616513874425, 'eval_balanced_accuracy': 0.6237030825693721, 'eval_runtime': 49.3725, 'eval_samples_per_second': 114.517, 'eval_steps_per_second': 7.17, 'epoch': 9.63}


Model weights saved in ./model\checkpoint-3400\pytorch_model.bin
tokenizer config file saved in ./model\checkpoint-3400\tokenizer_config.json
Special tokens file saved in ./model\checkpoint-3400\special_tokens_map.json
100%|██████████| 3530/3530 [1:40:49<00:00,  1.49s/it]

Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./model\checkpoint-600 (score: 0.6757269279393172).
100%|██████████| 3530/3530 [1:40:49<00:00,  1.71s/it]

{'train_runtime': 6049.7342, 'train_samples_per_second': 37.383, 'train_steps_per_second': 0.583, 'train_loss': 0.2583849422992458, 'epoch': 10.0}


TrainOutput(global_step=3530, training_loss=0.2583849422992458, metrics={'train_runtime': 6049.7342, 'train_samples_per_second': 37.383, 'train_steps_per_second': 0.583, 'train_loss': 0.2583849422992458, 'epoch': 10.0})

In [22]:
model.save_pretrained("./model/first_model")

Configuration saved in ./model/first_model\config.json
Model weights saved in ./model/first_model\pytorch_model.bin
